In [115]:

# %pip install selenium

In [46]:
# from datetime import datetime
# from datetime import timedelta
# from dateutil import relativedelta

# year_list = [i for i in range(2005, 2025)]
# month_list = [i for i in range(1, 13)]

# start_date_list = []
# end_date_list = []

# def getMonthRage(year, month):
#     this_month = datetime(year=year, month=month, day=1).date()
#     next_month = this_month + relativedelta.relativedelta(months=1)

#     first_day = this_month
#     last_day = next_month - timedelta(days=1)
    
#     start_date_list.append(str(first_day))
#     end_date_list.append(str(last_day))

# for year in year_list:
#     for month in month_list:
#         getMonthRage(year, month)


In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import pandas as pd
announce_dates_raw = pd.read_csv("C:/Users/SesacPython/Downloads/기준금리날짜.csv") 

In [2]:
announce_dates_raw

,announce_date,start,end
0,2005-02-15,2005-01-14,2005-02-14
1,2005-03-10,2005-02-16,2005-03-09
2,2005-04-07,2005-03-11,2005-04-06
3,2005-05-12,2005-04-08,2005-05-11
4,2005-06-09,2005-05-13,2005-06-08
...,...,...,...
204,2024-05-23,2024-04-13,2024-05-22
205,2024-07-11,2024-05-24,2024-07-10
206,2024-08-22,2024-07-12,2024-08-21
207,2024-10-11,2024-08-23,2024-10-10


In [5]:
full_announce_dates = announce_dates_raw.iloc[:206]

announce_dates = full_announce_dates.iloc[106:]
announce_dates

,announce_date,start,end
106,2013-11-14,2013-10-11,2013-11-13
107,2013-12-12,2013-11-15,2013-12-11
108,2014-01-09,2013-12-13,2014-01-08
109,2014-02-13,2014-01-10,2014-02-12
110,2014-03-13,2014-02-14,2014-03-12
...,...,...,...
201,2024-01-11,2023-12-01,2024-01-10
202,2024-02-22,2024-01-12,2024-02-21
203,2024-04-12,2024-02-23,2024-04-11
204,2024-05-23,2024-04-13,2024-05-22


In [3]:
announce_dates[announce_dates.announce_date=="2013-11-14"]

,announce_date,start,end
106,2013-11-14,2013-10-11,2013-11-13


In [6]:
date_lists = []
news_lists = []
final_news_links = []
err_list = []
hdr = {
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"
    }

for announce_date, start_date, end_date in zip(announce_dates.announce_date, announce_dates.start, announce_dates.end):

    testurl = f"https://www.mk.co.kr/search/news?word=%EA%B8%88%EB%A6%AC&dateType=direct&startDate={start_date}&endDate={end_date}"
    driver = webdriver.Chrome()
    driver.get(testurl)

    try:
        # '더보기' 버튼이 발견되고 클릭 가능한 동안 계속 반복
        while True:
            try:
                # '더보기' 버튼이 클릭 가능할 때까지 대기
                wait = WebDriverWait(driver, 10)
                more_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'btn.basic.w_f.drop_sub_news_btn.h_s.ic_before')))
                
                # '더보기' 버튼 클릭
                driver.execute_script("arguments[0].click();", more_button)
                
                # 서버에 과부하를 주지 않도록 작은 지연을 추가
                time.sleep(2)
            
            except TimeoutException:
                # 버튼이 지정된 시간 내에 발견되지 않으면 반복문 종료
                print("더 이상 클릭 가능한 '더보기' 버튼이 없습니다.")
                break
            
    except Exception as e:
        print(f"오류 발생: {e}")

    # =====================================================================

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    news_items =soup.select("a.news_item")
    news_links = []

    
    for i in news_items:
        try:
            href = i.attrs['href']
            news_links.append(href)
        except Exception as e:
            print(f"error link: {href}")
    # 각 링크별 본문 내용 수집

    
    for news_link in news_links:
        try:
        
            # 기사 내용
            res = requests.get(news_link, headers=hdr)
            soup = BeautifulSoup(res.text, 'html.parser')
            news_contents = soup.select(".news_cnt_detail_wrap")
            
            # 구독자 전용 기사 pass
            if news_contents == []:
                continue
            
            # 기사 날짜
            res2 = requests.get(news_link, headers=hdr)
            soup2 = BeautifulSoup(res2.text, 'html.parser')
            news_date_raw = soup.select("dl.registration > dd")
                
                
            news_text = news_contents[0].get_text()
            news_date = news_date_raw[0].get_text()
            news_lists.append(news_text)
            # final_news_links.append(news_link)
                
            date_lists.append(announce_date)
            # date_lists.append(news_date)
        except Exception as e:
            err_list.append(news_date)

    # news_dict[announce_date] = [[], []]
    # news_dict[announce_date][0] = news_lists
    # news_dict[announce_date][1] = final_news_links


df = pd.DataFrame({'news_lists':news_lists}, index=date_lists)
df.to_csv("C:/Users/SesacPython/Desktop/workspace/Team2/hong/mk_news_debug.csv")


더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼

In [5]:
df = pd.DataFrame({'news_lists':news_lists}, index=date_lists)
df.to_csv("C:/Users/SesacPython/Desktop/workspace/Team2/hong/mk_news_final.csv")

In [12]:
date_lists

[]

In [13]:
df.loc["2013-11-14"]

,news_lists
2013-11-14,\n아시아 주요국 증시가 미국 연방준비제도(Fed)의 연내 출구전략 시행 우려와 중...
2013-11-14,"\n""미국 연방준비제도의 양적완화 축소는 빨라야 내년 3월에 시작될 것으로 예상한다..."
2013-11-14,"\n\n\n블랙록 회장, 뉴욕 콘퍼런스서 전망""한국 5년간 年2.4% 성장""\n\n..."
2013-11-14,\n매일경제 전국 순회 투자설명회가 14일 대구에서 열린다. 이날 오후 2시부터 5...
2013-11-14,\n\n\n美 막대한 돈 풀어 경기부양 투자자금 우리나라로도 유입…일정기간 지나자 ...
2013-11-14,\n\n\n경제이슈 `콕콕` 취업역량 `쑥쑥`\n\n\n 사진 확대 ...
2013-11-14,\n 사진 확대 기대수명이 80세를 넘어서며 백(百)에서 일(一)...
2013-11-14,\n#.직장인 A씨(32·금호동)는 요즘 증시가 상승세를 보이자 주가연계증권(ELS...
2013-11-14,\n\n\nr114\n\n\nQ = 연말까지 집을 사면 다양한 세제혜택이 주어지고 ...
2013-11-14,\n\n\nr114\n\n\nQ = 연말까지 집을 사면 다양한 세제혜택이 주어지고 ...


In [18]:
pd.read_csv("C:/Users/SesacPython/Desktop/workspace/Team2/hong/mk_news_201311.csv", index_col=0)

,news_lists
2005-02-15,\n내전으로 폐허가 된 레바논의 재건사업을\n진두지휘했던 라피크 하리리(60) ...
2005-02-15,\n ========================================\...
2005-02-15,\n 다음달 5.5대 1 감자를 앞두고 있는 LG카드가 급등했다.\n\n 1...
2005-02-15,"\n ""지수 1000선 넘을 수 있을까.""\n\n 종합주가지수가 5년래 최고..."
2005-02-15,\n ◆달아오른 증시 960 훌쩍◆\n\n 14일 주가가 급등한 배경은 무엇...
...,...
2013-11-14,\n오는 14일 한국은행 금융통화위원회 예정된 가운데 채권전문가 대부부은 11월 기...
2013-11-14,\n국내 은행들의 올해 상반기 해외 점포 순이익도 작년보다 15% 줄어들었다.\n ...
2013-11-14,\n올해 상반기 국내은행 해외영업점의 수익성이 악화된 것으로 나타났다.\n 12...
2013-11-14,\n필리핀이 태풍 '하이옌'의 여파로 입은 경제적 피해가 최대 140억달러(약 15...


In [9]:
pd.read_csv("C:/Users/SesacPython/Desktop/workspace/Team2/hong/mk_news_debug.csv", index_col = 0)

,news_lists
2013-11-14,\n아시아 주요국 증시가 미국 연방준비제도(Fed)의 연내 출구전략 시행 우려와 중...
2013-11-14,"\n""미국 연방준비제도의 양적완화 축소는 빨라야 내년 3월에 시작될 것으로 예상한다..."
2013-11-14,"\n\n\n블랙록 회장, 뉴욕 콘퍼런스서 전망""한국 5년간 年2.4% 성장""\n\n..."
2013-11-14,\n매일경제 전국 순회 투자설명회가 14일 대구에서 열린다. 이날 오후 2시부터 5...
2013-11-14,\n\n\n美 막대한 돈 풀어 경기부양 투자자금 우리나라로도 유입…일정기간 지나자 ...
...,...
2024-07-11,\n 사진 확대 \n중산층이 겪는 경제적 어려움을 짚은 '위기의 중...
2024-07-11,\n 사진 확대 [사진 출처 = 연합뉴스] 서울 아파트 전세난이 월...
2024-07-11,\n 사진 확대 제롬 파월 미국 연방준비제도(Fed·연준) 의장 ...
2024-07-11,\n 사진 확대 [사진 출처 = 연합뉴스] 올해 1분기(1~3월) ...


In [1]:
import pandas as pd

In [5]:
final_df = pd.read_csv("C:/Users/SesacPython/Desktop/workspace/Team2/hong/mk_newslists_final.csv", index_col=0)

In [8]:
final_df

,news_lists
2005-02-15,\n내전으로 폐허가 된 레바논의 재건사업을\n진두지휘했던 라피크 하리리(60) ...
2005-02-15,\n ========================================\...
2005-02-15,\n 다음달 5.5대 1 감자를 앞두고 있는 LG카드가 급등했다.\n\n 1...
2005-02-15,"\n ""지수 1000선 넘을 수 있을까.""\n\n 종합주가지수가 5년래 최고..."
2005-02-15,\n ◆달아오른 증시 960 훌쩍◆\n\n 14일 주가가 급등한 배경은 무엇...
...,...
2024-07-11,\n 사진 확대 \n중산층이 겪는 경제적 어려움을 짚은 '위기의 중...
2024-07-11,\n 사진 확대 [사진 출처 = 연합뉴스] 서울 아파트 전세난이 월...
2024-07-11,\n 사진 확대 제롬 파월 미국 연방준비제도(Fed·연준) 의장 ...
2024-07-11,\n 사진 확대 [사진 출처 = 연합뉴스] 올해 1분기(1~3월) ...
